In [1]:
!pip install BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
# 만개의 매거진 Main 레시피 id 크롤링
def MainRecipeIdCrawler(pageId):

    # for i in range(1, pageId + 1):
    # url = f'https://www.10000recipe.com/issue/view.html?cid=10000mag&page={i}'

    url = 'https://www.10000recipe.com/issue/view.html?cid=10000mag&page=' + pageId

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    main_recipe_id = [] # Main 레시피 id

    # Main 레시피 id
    elements = soup.select('#contents_area_full > div.chef_cont > div > div > a.thumbnail')
    for element in elements:
        id = re.sub(r'\D', '', element['href'])
        main_recipe_id.append(id)

    return(main_recipe_id)

In [4]:
# 만개의 매거진 1페이지
MainRecipeIdCrawler(str(1))

['6988334',
 '6987988',
 '6987503',
 '6987032',
 '6986287',
 '6985914',
 '6985381',
 '6984838',
 '6984384',
 '6983991',
 '6983544',
 '6983029',
 '6982533',
 '6982076',
 '6981536',
 '6980953',
 '6980539',
 '6980076',
 '6979563',
 '6979200']

In [5]:
# 만개의 매거진 Sub 레시피 id 크롤링
def SubRecipeIdCrawler(main_recipe_id):

    url = 'https://www.10000recipe.com/recipe/' + main_recipe_id

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    sub_recipe_id = [] # Sub 레시피 id

    # Sub 레시피 id 및 이름
    elements = soup.select('#contents_area > div > div.view_step_cont')
    for element in elements:
        id = re.sub(r'\D', '', element.find('a')['href'])
        sub_recipe_id.append(id)

    return(sub_recipe_id)

In [6]:
# 도시락에 빠질 수 없는 ✿유부초밥✿ 레시피 모음 (연어유부초밥, 크래미유부초밥, 명란마요유부초밥, 새우유부밥, 삼색유부초밥)
SubRecipeIdCrawler(str(6988334))

['6909678', '6916853', '6984195', '6962138', '6855278']

In [7]:
# 레시피 하나만 크롤링
def RecipeCrawler(recipeId):
    url = 'https://www.10000recipe.com/recipe/' + recipeId

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    recipe_id = [] # 레시피 id
    recipe_name = [] # 레시피 이름
    recipe_thumbnail = [] # 레시피 썸네일
    recipe_time = [] # 레시피 시간
    recipe_difficulty = [] # 레시피 난이도
    recipe_composition = [] # 레시피 구성
    recipe_ingredients = [] # 레시피 재료
    recipe_seasoning = [] # 레시피 양념
    recipe_orders = [] # 레시피 조리 순서
    recipe_photo = [] # 레시피 사진

    # 레시피 id
    recipe_id.append(recipeId)

    # 레시피 이름
    name = soup.select_one('#relationGoods > div.best_tit')
    recipe_name.append(name.get_text().split()[0])

    # 레시피 썸네일
    thumbnail = soup.select_one('#contents_area > div.view2_pic > div.centeredcrop img')['src']
    recipe_thumbnail.append(thumbnail)

    # 레시피 시간
    time = soup.select_one('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2').get_text()
    recipe_time.append(time)

    # 레시피 난이도
    difficulty = soup.select_one('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3').get_text()
    recipe_difficulty.append(difficulty)

    # 레시피 구성 (정보 없음)
    recipe_composition.append('든든하게')

    # 레시피 재료
    elements = soup.select_one('#divConfirmedMaterialArea > ul:nth-child(1)')
    tmp = ''
    for element in elements.select('li'):
        ingredient = re.sub(r'\s+', ' ', element.get_text().replace('구매', '').strip()) # 구매 링크 제외
        tmp = tmp + ('#' + ingredient)
    recipe_ingredients.append(tmp)
    
    # 레시피 양념
    try:
        elements = soup.select_one('#divConfirmedMaterialArea > ul:nth-child(2)')
        tmp = ''
        for element in elements.select('li'):
            seasoning = re.sub(r'\s+', ' ', element.get_text().replace('구매', '').strip()) # 구매 링크 제외
            tmp = tmp + ('#' + seasoning)
        recipe_seasoning.append(tmp)
    except AttributeError:
        recipe_seasoning.append('')
    else:
        pass

    # 레시피 조리 순서 및 사진
    elements = soup.select_one('#contents_area > div:nth-child(16)')
    elements = elements.find_all('div', 'view_step_cont')
    tmp1 = ''
    tmp2 = ''
    for i, element in enumerate(elements, start=1):
        order = element.find('div', 'media-body')
        order = ''.join(order.findAll(string=True, recursive=False)).strip() # 조리 순서에서 설명, 주방 도구 제외
        tmp1 = tmp1 + ('#' + str(i) + '. ' + order)
        photo = element.find('img')['src']
        tmp2 = tmp2 + ('#' + photo)
    recipe_orders.append(tmp1)
    recipe_photo.append(tmp2)
                                 
    recipe_all = [recipe_id, recipe_name, recipe_thumbnail, recipe_time, recipe_difficulty, recipe_composition, recipe_ingredients, recipe_seasoning, recipe_orders, recipe_photo]
    
    return(recipe_all)

In [8]:
# 연어유부초밥 (만개의 매거진의 레시피)
RecipeCrawler(str(6909678))

[['6909678'],
 ['연어유부초밥'],
 ['https://recipe1.ezmember.co.kr/cache/recipe/2019/04/02/f8e2bac1e4e5387b34ef9dfa04f343b41.jpg'],
 ['15분 이내'],
 ['아무나'],
 ['든든하게'],
 ['#시판 유부 초밥 1봉지#밥 1+1/2공기#연어 1팩'],
 ['#다진 양파 2숟가락#마요네즈 3가락#레몬즙 1/2숟가락#꿀 1숟가락#소금 약간#후추 약간'],
 ['#1. 연어는 작게 썬다.#2. 볼에 소스를 만든다.#3. 밥에 배합초를 넣고 버무린다.#4. 유부의 물기를 짜고 밥>소스>연어를 올려 완성한다.'],
 ['#https://recipe1.ezmember.co.kr/cache/recipe/2019/04/01/1617be01e2c10f03acd3dde68c6f954f1.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2019/04/01/0d63f0109b119a2b19d657de551ac10f1.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2019/04/01/d576fafdfe011986ea97ea2c2f6040311.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2019/04/01/c3be7cecabd170ada406bd7ee65362db1.jpg']]

In [9]:
# 크래미유부초밥 (만개의 매거진의 레시피)
RecipeCrawler(str(6916853))

[['6916853'],
 ['크래미유부초밥'],
 ['https://recipe1.ezmember.co.kr/cache/recipe/2019/08/01/ccc38222338c8673dc9588c610303a811.jpg'],
 ['20분 이내'],
 ['아무나'],
 ['든든하게'],
 ['#크래미 5개#밥 2/3공기#시판 유부초밥 1봉지#양파 1/4개'],
 ['#허니머스터드 1숟가락#마요네즈 3+1/2숟가락#후추 약간'],
 ['#1. 양파는 잘게 다진 뒤 찬물에 5분간 담가 건져요.#2. 크래미를 결대로 찢은 후 양파, 양념 재료를 넣고 섞어요.#3. 밥에 유부초밥 후리가케와 배합초를 넣고 섞어요.#4. 유부는 물기를 짠 후 밥을 2/3정도 채워 주고 양념한 크래미를 올려 완성해요.'],
 ['#https://recipe1.ezmember.co.kr/cache/recipe/2019/08/01/60c14bbde877d3d236da5f8e73150c981.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2019/08/01/42d42653f516f12fa7120a8563f5918c1.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2019/08/01/473bdc6adc55aec411fb7451b037ce071.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2019/08/01/c54c2323729a60f4a1076b9a0a5d32701.jpg']]

In [10]:
# 치즈케이크 (만개의 레시피의 레시피)
RecipeCrawler(str(6992105))

[['6992105'],
 ['치즈케이크'],
 ['https://recipe1.ezmember.co.kr/cache/recipe/2022/11/16/fb0b4b6803b6e4c0f3f466f099f033b81.jpg'],
 ['20분 이내'],
 ['아무나'],
 ['든든하게'],
 ['#크림치즈 200g#설탕 3숟가락#달걀 노른자 2개#밀가루 박력분 2숟가락#바닐라 익스트랙 1~2방울#식용유 약간'],
 [''],
 ['#1. 크림치즈를 부드럽게 풀어준다.#2. 설탕을 넣고 설탕이 녹을 때까지 잘 풀어준다.#3. 달걀 노른자를 넣고 덩어리가 생기지 않도록 풀어준다.#4. 밀가루 박력분을 체 쳐 넣고 섞는다.#5. 바닐라 익스트랙을 넣고 섞는다.#6. 전자레인지용 용기에 식용유를 살짝 바르고 반죽을 붓는다.#7. 평평하게 만든 후 뚜껑을 덮고 전자레인지에 넣어 4분(700W 기준) 돌린다.'],
 ['#https://recipe1.ezmember.co.kr/cache/recipe/2022/11/16/6d2099d4041a97a4bb2b301ce06905911.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2022/11/16/35a6c38fe87ea9739872ff31a239d0201.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2022/11/16/cabb4e7cf3dc067e730aa8f9c9b1d1231.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2022/11/16/921e0b136528411239767ddce2c6e6fc1.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2022/11/16/639dd4d7a7ef0287f7a8e7761db05ca11.jpg#https://recipe1.ezmember.co.kr/cache/recipe/2022/11/16/2e27d822236f43775ce3239e